# Week 2 GRAPPA Seminar Assignment

In [1]:
import numpy as np
import scipy.integrate
import matplotlib.pyplot as plt
import time

Create a cell that converts all the units into natural units.

In [2]:
# length
cm = 1
m = 1e2 * cm
km = 1e3 * m
pc = 3.1e18 * cm
Mpc = 1e6 * pc
kpc = 1e3 * pc

# time
s = 1

# mass/energy
GeV = 1
eV = 1e-9 * GeV
kg = 5.61e26 * GeV
M_solar = 2e30 * kg

## 1. What are the estimates of r$_{s}$ and $\rho_{s}$ for the Milky-Way Halo?

The Milky-Way halo has a mass of around 10$^{12}$ solar masses.

In [3]:
M = 1e12*M_solar
a_0 = 1
h = 0.7
H_0 = h * 100 * km * s**(-1) * Mpc**(-1)
Omega_m0 = 0.32
Omega_Lambda = 0.68
c_vir = 10

Define functions to calculate $\Omega_{\mathrm{m}}$, the Hubble constant H, $\Delta_{\mathrm{vir}}$, $\rho_{\mathrm{critical}}$, and r$_{\mathrm{vir}}$ in order to obtain values for r$_{s}$ and $\rho_{s}$.

In [4]:
def Omega_m(z):
    return Omega_m0*(1+z)**3 / (Omega_m0*(1+z)**3 + Omega_Lambda)

def H(z):
    return np.sqrt(H_0**2 * (Omega_m(z) + Omega_Lambda))

def delta_vir(z):
    return 18*np.pi**2 + 82*(Omega_m(z) - 1) - 39*(Omega_m(z) - 1)**2

def rho_c(z):
    return 2.775e11*h**2*M_solar/Mpc**3

def r_vir(z):
    return (3*M)**(1/3) / (4*np.pi*delta_vir(z)*rho_c(z))**(1/3)

def r_s(z):
    return r_vir(z) / c_vir

def rho_s(z):
    denominator = np.log(1 + c_vir) - (c_vir/(1 + c_vir))
    return M / (denominator*4*np.pi*r_s(z)**3)

In [5]:
print(r_s(0)/kpc, rho_s(0))

25.664669324262622 0.1190839575972599


## 2. How does $dJ/dΩ$ look like as a function of angle subtending from the Galactic center $\psi$?

In [6]:
r_s0 = 20*kpc
rho_s0 = 0.3

d_gc = 8*kpc

l_max = 1e2*kpc
z = 0

In [7]:
def rho(l, z, subt_angle, theta):
#     ANGLES IN RADIANS
    
    r    = np.sqrt(-2*l*d_gc*np.cos(subt_angle)*np.cos(theta) + d_gc**2 + l**2)

#     return rho_s(z)/((r/r_s(z))*(r/r_s(z)+1)**2)
    return rho_s0/((r/r_s0)*(r/r_s0+1)**2)

In [8]:
rho_2 = lambda l, z, subt_angle, theta: rho(l, z, subt_angle, theta)**2

In [9]:
def dJdΩ_riemann(subt_angle,theta):
    z = 0
    n_values = 1000
    
    l_values = np.linspace(0, l_max, n_values)
    dJdΩ_func = rho_2((l_values[1:]+l_values[:-1]),z,subt_angle,theta)*(l_values[1:]-l_values[:-1])/2
    return np.sum(dJdΩ_func)


n_grid = 100
ang_max_deg = 0.5

ang_max = ang_max_deg * (2*np.pi/360)
xang_grid = np.linspace(-ang_max, ang_max, n_grid)
yang_grid = np.linspace(-ang_max, ang_max, n_grid)


J_arr = np.array([])
for theta in xang_grid:
    for phi in yang_grid:
        if (np.sin(theta)**2 + np.sin(phi)**2 <= np.sin(ang_max)**2):
            J_arr = np.append(J_arr, dJdΩ_riemann(phi, theta))
            
J_factor = np.mean(J_arr)
print(J_factor)

1.1184683923790519e+24
